In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from TO_sim.Sol_Kuramoto import Sol_Kuramoto_mf2 as mf2
from TO_sim.Sol_Kuramoto import Sol_Kuramoto_mf2_sets as mf2_sets
from TO_sim.get_cluster import C_rsmso,cluster_os,C_rsmso_set_new
from TO_sim.To_Draw import Draw_avg_vel_r

from TO_sim.gen_Distribution import Normal
from TO_sim.gen_Distribution import Quantile_Normal as Q_Normal
import parmap
from scipy.signal import find_peaks
from TO_sim.To_Draw import Draw_avg_vel_r
from tqdm.notebook import tqdm


In [2]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

In [19]:
dK = 0.1
dm = 0.1
N = 1000
seed = 10
dt = 0.1
theta_init,omega,Kc = Q_Normal(N,0,1,seed=seed)
K_set = np.arange(0,5,dK).reshape(-1,1)

N_set = len(K_set)
omega = np.sort(omega) - np.mean(omega)
ms = np.arange(0,10+dm/2,dm)
m_set = []
core = 8
index_ = 0
for i in range(len(ms)//core +1):
    m_set.append(ms[index_:index_+core])
    index_ += core
core_set = [len(ms) for ms in m_set]
N_set = len(K_set)
theta_init_set = (np.ones((N_set,1))*theta_init)
pdtheta_set = 0*(np.ones((N_set,1))*omega)
omega_set = (np.ones((N_set,1))*omega)

In [4]:
core_set

[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 5]

In [20]:
cols ={'r':[0],'rstd':[0],'rMM':[0],'error':[0]}
for i in range(10):
    cols[f'c{i}'] = np.nan
for i in range(10):
    cols[f'c{i} phase vel'] = np.nan
for i in range(10):
    cols[f'c{i} omega'] = np.nan

cols['K'] = K_set.reshape(-1)[0]
cols['m'] = ms[0]
df = pd.DataFrame(cols)
df_Km = df.set_index(['K','m'])

In [21]:
for core,ms in zip(core_set,m_set):
    for m,(r,rstd,rMM,CSMP_set,CSMO_set,error_set,C_s_set,C_e_set) in zip(ms,parmap.map(C_rsmso_set_new,ms,K_set=K_set,N=N,theta_init_set=theta_init_set,
                                                        omega_set=omega_set,pdtheta_set = pdtheta_set,t_end=3000,dt=dt,
                                                        pm_pbar=True, pm_processes=core)):
        for i,K in enumerate(K_set.reshape(-1)):
            df_Km.loc[(K,m),"r"] = r[i]
            df_Km.loc[(K,m),"rstd"] = rstd[i]
            df_Km.loc[(K,m),"rMM"] = rMM[i]
            df_Km.loc[(K,m),'error'] = error_set[i]
            CSMP = CSMP_set[i]
            CSMO = CSMO_set[i]
            for i in range(len(CSMP_set[i])):
                    df_Km.loc[(K,m),f'c{i}'] = CSMP['cluster size'][i]
                    df_Km.loc[(K,m),f'c{i} phase vel'] = CSMP['cluster mean phase velocity'][i]
                    df_Km.loc[(K,m),f'c{i} omega'] = CSMO['cluster mean natural frequency'][i]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

In [17]:
df_Km.sort()

AttributeError: 'DataFrame' object has no attribute 'sort'

In [6]:
K_set = np.arange(5,10+dK/2,dK).reshape(-1,1)
N_set = len(K_set)
omega = np.sort(omega) - np.mean(omega)
ms = np.arange(0,10+dm/2,dm)
m_set = []
core = 8
index_ = 0
for i in range(len(ms)//core +1):
    m_set.append(ms[index_:index_+core])
    index_ += core
core_set = [len(ms) for ms in m_set]
N_set = len(K_set)
theta_init_set = (np.ones((N_set,1))*theta_init)
pdtheta_set = 0*(np.ones((N_set,1))*omega)
omega_set = (np.ones((N_set,1))*omega)
for core,ms in zip(core_set,m_set):
    for m,(r,rstd,rMM,CSMP_set,CSMO_set,error_set,C_s_set,C_e_set) in zip(ms,parmap.map(C_rsmso_set_new,ms,K_set=K_set,N=N,theta_init_set=theta_init_set,
                                                        omega_set=omega_set,pdtheta_set = pdtheta_set,t_end=2000,dt=dt,
                                                        pm_pbar=True, pm_processes=core)):
        for i,K in enumerate(K_set.reshape(-1)):
            df_Km.loc[(K,m),"r"] = r[i]
            df_Km.loc[(K,m),"rstd"] = rstd[i]
            df_Km.loc[(K,m),"rMM"] = rMM[i]
            df_Km.loc[(K,m),'error'] = error_set[i]
            CSMP = CSMP_set[i]
            CSMO = CSMO_set[i]
            for i in range(len(CSMP_set[i])):
                    df_Km.loc[(K,m),f'c{i}'] = CSMP['cluster size'][i]
                    df_Km.loc[(K,m),f'c{i} phase vel'] = CSMP['cluster mean phase velocity'][i]
                    df_Km.loc[(K,m),f'c{i} omega'] = CSMO['cluster mean natural frequency'][i]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

...

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [18]:
f'N = {N} seed = {seed} dt = {dt} 230708 ver3.xlsm'

'N = 500 seed = 10 test dt = 0.1 230708 ver3.xlsm'

In [18]:
df_Km.sort_index()

r      rstd       rMM  error     c0    c1    c2  c3  c4  c5  \
K    m                                                                          
0.0  0.0   0.042354  0.018785  0.091731    NaN    NaN   NaN   NaN NaN NaN NaN   
     0.1   0.042346  0.018787  0.091731    NaN    NaN   NaN   NaN NaN NaN NaN   
     0.2   0.042334  0.018789  0.091731    NaN    NaN   NaN   NaN NaN NaN NaN   
     0.3   0.042320  0.018792  0.091731    NaN    NaN   NaN   NaN NaN NaN NaN   
     0.4   0.042306  0.018794  0.091731    NaN    NaN   NaN   NaN NaN NaN NaN   
...             ...       ...       ...    ...    ...   ...   ...  ..  ..  ..   
10.0 9.6   0.806193  0.062233  0.197309    NaN  417.0  28.0  22.0 NaN NaN NaN   
     9.7   0.801457  0.063791  0.203774    NaN  415.0  28.0  23.0 NaN NaN NaN   
     9.8   0.807222  0.062646  0.204616    NaN  417.0  28.0  22.0 NaN NaN NaN   
     9.9   0.804293  0.062826  0.205518    NaN  416.0  29.0  21.0 NaN NaN NaN   
     10.0  0.807415  0.060879  0.198382    NaN  417.0  28.0  21.0 NaN NaN NaN   

           ...  c0 omega  c1 omega  c2 omega  c3 omega  c4 omega  c5 omega  \
K    m     ...                                                               
0.0  0.0   ...       NaN       NaN       NaN       NaN       NaN       NaN   
     0.1   ...       NaN       NaN       NaN       NaN       NaN       NaN   
     0.2   ...       NaN       NaN       NaN       NaN       NaN       NaN   
     0.3   ...       NaN       NaN       NaN       NaN       NaN       NaN   
     0.4   ...       NaN       NaN       NaN       NaN       NaN       NaN   
...        ...       ...       ...       ...       ...       ...       ...   
10.0 9.6   ... -0.040027  1.535441 -1.577685       NaN       NaN       NaN   
     9.7   ... -0.040027  1.504645 -1.569878       NaN       NaN       NaN   
     9.8   ... -0.040027  1.535441 -1.577685       NaN       NaN       NaN   
     9.9   ... -0.040027  1.513667 -1.567756       NaN       NaN       NaN   
     10.0  ... -0.040027  1.504645 -1.593635       NaN       NaN       NaN   

           c6 omega  c7 omega  c8 omega  c9 omega  
K    m                                             
0.0  0.0        NaN       NaN       NaN       NaN  
     0.1        NaN       NaN       NaN       NaN  
     0.2        NaN       NaN       NaN       NaN  
     0.3        NaN       NaN       NaN       NaN  
     0.4        NaN       NaN       NaN       NaN  
...             ...       ...       ...       ...  
10.0 9.6        NaN       NaN       NaN       NaN  
     9.7        NaN       NaN       NaN       NaN  
     9.8        NaN       NaN       NaN       NaN  
     9.9        NaN       NaN       NaN       NaN  
     10.0       NaN       NaN       NaN       NaN  

[10201 rows x 34 columns]

In [15]:
df_Km.sort_index().to_excel(f'N = {N} Q_Normal seed = {seed} dt = {dt} 230821 ver3.xlsm')

In [8]:
f'N = {N} Q_Normal seed = {seed} dt = {dt} 230821 ver3.xlsm'

'N = 500 Q_Normal seed = 10 dt = 0.1 230821 ver3.xlsm'

In [10]:
f'N = {N} seed = {seed} dt = {dt} 230720 ver3.xlsm'

'N = 500 seed = 10 dt = 0.1 230720 ver3.xlsm'

In [ ]:
for seed in [10]:
    dK = 0.1
    dm = 0.1
    N = 500

    theta_init,omega,Kc = Normal(N,0,1,seed=seed)
    K_set = np.arange(0,5,dK).reshape(-1,1)

    N_set = len(K_set)
    omega = np.sort(omega) - np.mean(omega)
    ms = np.arange(0,10+dm/2,dm)
    m_set = []
    core = 8
    index_ = 0
    for i in range(len(ms)//core +1):
        m_set.append(ms[index_:index_+core])
        index_ += core
    core_set = [len(ms) for ms in m_set]
    N_set = len(K_set)
    theta_init_set = (np.ones((N_set,1))*theta_init)
    pdtheta_set = 0*(np.ones((N_set,1))*omega)
    omega_set = (np.ones((N_set,1))*omega)
    core_set
    cols ={'r':[0],'rstd':[0],'rMM':[0],'error':[0]}
    for i in range(10):
        cols[f'c{i}'] = np.nan
    for i in range(10):
        cols[f'c{i} phase vel'] = np.nan
    for i in range(10):
        cols[f'c{i} omega'] = np.nan

    cols['K'] = K_set.reshape(-1)[0]
    cols['m'] = ms[0]
    df = pd.DataFrame(cols)
    df_Km = df.set_index(['K','m'])
    for core,ms in tqdm(list(zip(core_set,m_set))):pass
    for core,ms in zip(core_set,m_set):
        for m,(r,rstd,rMM,CSMP_set,CSMO_set,error_set,C_s_set,C_e_set) in zip(ms,parmap.map(C_rsmso_set_new,ms,K_set=K_set,N=N,theta_init_set=theta_init_set,
                                                            omega_set=omega_set,pdtheta_set = pdtheta_set,t_end=2000,dt=0.1,
                                                            pm_pbar=True, pm_processes=core)):
            for i,K in enumerate(K_set.reshape(-1)):
                df_Km.loc[(K,m),"r"] = r[i]
                df_Km.loc[(K,m),"rstd"] = rstd[i]
                df_Km.loc[(K,m),"rMM"] = rMM[i]
                df_Km.loc[(K,m),'error'] = error_set[i]
                CSMP = CSMP_set[i]
                CSMO = CSMO_set[i]
                for i in range(len(CSMP_set[i])):
                        df_Km.loc[(K,m),f'c{i}'] = CSMP['cluster size'][i]
                        df_Km.loc[(K,m),f'c{i} phase vel'] = CSMP['cluster mean phase velocity'][i]
                        df_Km.loc[(K,m),f'c{i} omega'] = CSMO['cluster mean natural frequency'][i]
    K_set = np.arange(5,10+dK/2,dK).reshape(-1,1)
    N_set = len(K_set)
    omega = np.sort(omega) - np.mean(omega)
    ms = np.arange(0,10+dm/2,dm)
    m_set = []
    core = 8
    index_ = 0
    for i in range(len(ms)//core +1):
        m_set.append(ms[index_:index_+core])
        index_ += core
    core_set = [len(ms) for ms in m_set]
    N_set = len(K_set)
    theta_init_set = (np.ones((N_set,1))*theta_init)
    pdtheta_set = 0*(np.ones((N_set,1))*omega)
    omega_set = (np.ones((N_set,1))*omega)
    for core,ms in zip(core_set,m_set):
        for m,(r,rstd,rMM,CSMP_set,CSMO_set,error_set,C_s_set,C_e_set) in zip(ms,parmap.map(C_rsmso_set_new,ms,K_set=K_set,N=N,theta_init_set=theta_init_set,
                                                            omega_set=omega_set,pdtheta_set = pdtheta_set,t_end=2000,dt=0.1,
                                                            pm_pbar=True, pm_processes=core)):
            for i,K in enumerate(K_set.reshape(-1)):
                df_Km.loc[(K,m),"r"] = r[i]
                df_Km.loc[(K,m),"rstd"] = rstd[i]
                df_Km.loc[(K,m),"rMM"] = rMM[i]
                df_Km.loc[(K,m),'error'] = error_set[i]
                CSMP = CSMP_set[i]
                CSMO = CSMO_set[i]
                for i in range(len(CSMP_set[i])):
                        df_Km.loc[(K,m),f'c{i}'] = CSMP['cluster size'][i]
                        df_Km.loc[(K,m),f'c{i} phase vel'] = CSMP['cluster mean phase velocity'][i]
                        df_Km.loc[(K,m),f'c{i} omega'] = CSMO['cluster mean natural frequency'][i]
    df_Km.sort_index().to_excel(f'N = {N} seed = {seed} test 230708 ver3.xlsm')
